In [1]:
#Read data
import csv
import cv2
import numpy as np
import sklearn

lines =[]
#imagepath=[]
#measurements=[]

filepath='../CarND-Behavioral-Cloning-P3/data_track2/'
with open(filepath+'driving_log.csv') as csvfile:
    reader= csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        

for line in lines:
    center_source_path=line[0]
    left_source_path = line[1]
    right_source_path = line[2]
    center_filename=center_source_path.split('/')[-1]
    center_current_path=filepath+'IMG/'+center_filename
#     image = cv2.imread(center_current_path)
    imagepath.append(center_current_path)
    measurement = float(line[3])
    measurements.append(measurement)
    
    left_filename=left_source_path.split('/')[-1]
    left_current_path=filepath+'IMG/'+left_filename
#     image = cv2.imread(left_current_path)
    imagepath.append(left_current_path)
    measurement = float(line[3])
    measurements.append(measurement+0.2)
    
    right_filename=right_source_path.split('/')[-1]
    right_current_path=filepath+'IMG/'+right_filename
#     image = cv2.imread(right_current_path)
    imagepath.append(right_current_path)
    measurement = float(line[3])
    measurements.append(measurement-0.2)
    
from sklearn.model_selection import train_test_split
samples = list(zip(imagepath, measurements))
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: 
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for imagePath, measurement in batch_samples:
                originalImage = cv2.imread(imagePath)
                image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
                images.append(image)
                angles.append(measurement)

            # trim image to only see section with road
            inputs = np.array(images)
            outputs = np.array(angles)
            yield sklearn.utils.shuffle(inputs, outputs)
            
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [2]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, Cropping2D, Lambda
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

def preProcessing(model):
    'Normalization, mean-centring and cropping'
    model.add(Lambda(lambda x:x/255.0-0.5,input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    return model

def nVidiaModel(model):
    """
    Creates nVidea Autonomous Car Group model
    """
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

model = Sequential()
model = preProcessing(model)
model = nVidiaModel(model)
# model.add(Flatten(input_shape=(160,320,3)))
# model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(train_generator, samples_per_epoch= \
                 len(train_samples), validation_data=validation_generator, \
                 nb_val_samples=len(validation_samples), nb_epoch=3, verbose=1)

model.save('model.h5')
#model.save('model1.h5')